In [1]:
# @title Connect to gdrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive

In [ ]:
import os
def download_file(url):
    !wget {url} -O {os.path.basename(url)}
url = "https://example.com/files/example.zip"
download_file(url)

In [ ]:
# @title Extract audio track
#extract audio
import subprocess

def extract_audio(input_video, output_audio):
    command = [
        'ffmpeg',
        '-i', input_video,
        '-vn',  # Disable video recording
        '-acodec', 'copy',  # Copy audio codec without re-encoding
        #'-map', '0:a:1',  # Select the second audio stream (0-indexed)
        output_audio
    ]

    subprocess.run(command)

if __name__ == "__main__":
    input_video_file = "ben.mkv"
    output_audio_file = "ben2.mp4"
    for i in range(11,20):
        extract_audio(f"{i}.mkv", f"./nullvideo/{i}-vid.mp4")

In [ ]:
ls

In [ ]:
# @title Audio track count find
#audio track count find
import subprocess
import json

def get_all_audio_track_info(input_video):
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'a',
        '-show_entries', 'stream=index:stream_tags=language',
        '-of', 'json',
        input_video
    ]

    try:
        result = subprocess.run(command, capture_output=True, text=True)
        output = result.stdout

        data = json.loads(output)
        audio_tracks = [
            {"index": entry["index"], "language": entry["tags"].get("language", "unknown")}
            for entry in data.get("streams", [])
        ]

        return audio_tracks

    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        return None

if __name__ == "__main__":
    input_video_file = "ben.mkv"

    audio_track_info = get_all_audio_track_info(input_video_file)

    if audio_track_info is not None:
        print("All Audio Tracks Information:")
        for track in audio_track_info:
            print(f"Index: {track['index']}, Language: {track['language']}")
    else:
        print("Failed to retrieve audio track information.")

In [9]:
# @title Unzip a file
import patoolib

def unzip_file(file_path, extract_path):
    try:
        patoolib.extract_archive(file_path, outdir=extract_path)
        print(f"Successfully extracted {file_path} to {extract_path}")
    except Exception as e:
        print(f"Error extracting {file_path}: {e}")

# Example usage
compressed_file_path = '/content/drive/MyDrive/DriveSeed/okay.zip'
extract_path = '/content/drive/MyDrive/extrected'

unzip_file(compressed_file_path, extract_path)





INFO patool: Extracting /content/drive/MyDrive/DriveSeed/okay.zip ...
INFO:patool:Extracting /content/drive/MyDrive/DriveSeed/okay.zip ...
INFO patool: ... creating output directory `extrected'.
INFO:patool:... creating output directory `extrected'.
INFO patool: running /usr/bin/7z x -oextrected -- /content/drive/MyDrive/DriveSeed/okay.zip
INFO:patool:running /usr/bin/7z x -oextrected -- /content/drive/MyDrive/DriveSeed/okay.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /content/drive/MyDrive/DriveSeed/okay.zip extracted to `extrected'.
INFO:patool:... /content/drive/MyDrive/DriveSeed/okay.zip extracted to `extrected'.


Successfully extracted /content/drive/MyDrive/DriveSeed/okay.zip to extrected


In [4]:
pwd


'/content/drive/MyDrive'

In [ ]:
# @title h264 video(aac) => h264 video(mp3)
import subprocess

def copy_video_change_audio_to_mp3(input_video, output_video):
    command = [
        'ffmpeg',
        '-i', input_video,
        '-c:v', 'copy',    # Copy the video codec without re-encoding
        '-c:a', 'mp3',     # Set the audio codec to MP3
        '-strict', 'experimental',  # Needed for using AAC codec
        output_video
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Video copied, and audio codec changed to MP3. Saved to {output_video}.")
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    input_video_file = "path/to/your/input/video.mp4"
    output_video_file = "path/to/your/output/copied_video.mp4"
    for i in range(6,11):
        copy_video_change_audio_to_mp3(f"{i}.mkv", f"./processed/{i}.mp4")
